In [1]:
import pandas as pd

from base.training import mse_weighted
from common import Predictor, DataStorage, L2Threshold
from common.model_utils import load_model_tflite
from simulation.model_simulator import ModelSimulator
from simulation.model_simulator import VIOLATION_COLUMN

if mse_weighted:
    print("mse_weighted loaded")

df_baseline = pd.read_pickle("zamg_vienna_hourly.pickle")

features = ["TL", "P", "RF", "SO"]
df_baseline = df_baseline[features]
df_baseline.dropna(inplace=True)

df_2019_2019 = df_baseline[df_baseline.index.year == 2019]
df_2010_2019 = df_baseline[df_baseline.index.year <= 2019]
initial_df = df_2010_2019[features].asfreq('H')

sim_data: pd.DataFrame = df_baseline.loc[df_baseline.index.year >= 2020]

THRESHOLD = 1.0
threshold = L2Threshold(THRESHOLD, [0], [0])

sim_data

mse_weighted loaded


,TL,P,RF,SO
time,,,,
2020-01-01 00:00:00,5.0,1012.4,60.0,0.0
2020-01-01 00:10:00,5.0,1012.5,60.0,0.0
2020-01-01 00:20:00,4.9,1012.5,61.0,0.0
2020-01-01 00:30:00,4.9,1012.5,61.0,0.0
2020-01-01 00:40:00,4.9,1012.6,60.0,0.0
...,...,...,...,...
2021-12-31 23:10:00,15.3,999.2,59.0,0.0
2021-12-31 23:20:00,15.2,999.3,59.0,0.0
2021-12-31 23:30:00,15.2,999.3,59.0,0.0


In [2]:
# Vienna 2019-2019 simple dense
model = load_model_tflite("models/zamg_vienna_2019_2019_simple_dense")
metadata = model.metadata
data_storage = DataStorage(metadata.input_features, metadata.output_features)
data_storage.add_measurement_df(initial_df)
predictor = Predictor(model, data_storage)
predictor.update_prediction_horizon(sim_data.index.min())
model_simulator = ModelSimulator(predictor, threshold, sim_data, 600)

results_2019_2019 = model_simulator.run()
results_2019_2019

,TL,P,RF,SO,prediction,error,model,violation
2020-01-01 00:00:00,5.0,1012.4,60.0,0.0,5.1,0.1,zamg_vienna_2019_2019_simple_dense,0
2020-01-01 00:10:00,5.0,1012.5,60.0,0.0,4.882985,0.117015,zamg_vienna_2019_2019_simple_dense,0
2020-01-01 00:20:00,4.9,1012.5,61.0,0.0,4.66597,0.23403,zamg_vienna_2019_2019_simple_dense,0
2020-01-01 00:30:00,4.9,1012.5,61.0,0.0,4.448955,0.451045,zamg_vienna_2019_2019_simple_dense,0
2020-01-01 00:40:00,4.9,1012.6,60.0,0.0,4.23194,0.66806,zamg_vienna_2019_2019_simple_dense,0
...,...,...,...,...,...,...,...,...
2021-12-31 23:10:00,15.3,999.2,59.0,0.0,14.849399,0.450601,zamg_vienna_2019_2019_simple_dense,0
2021-12-31 23:20:00,15.2,999.3,59.0,0.0,14.791105,0.408895,zamg_vienna_2019_2019_simple_dense,0
2021-12-31 23:30:00,15.2,999.3,59.0,0.0,14.73281,0.46719,zamg_vienna_2019_2019_simple_dense,0
2021-12-31 23:40:00,15.0,999.7,59.0,0.0,14.674515,0.325485,zamg_vienna_2019_2019_simple_dense,0


In [3]:
violations_2019_2019 = results_2019_2019[results_2019_2019[VIOLATION_COLUMN] == True]
print(f"Simulation data: {results_2019_2019.index.min()} - {results_2019_2019.index.max()}")
print(f"Violations: {len(violations_2019_2019)}")
print(f"Measurements: {len(results_2019_2019)}")
print(f"Violations / Measurements = {len(violations_2019_2019) / len(results_2019_2019)}")

Simulation data: 2020-01-01 00:00:00 - 2021-12-31 23:50:00
Violations: 5861
Measurements: 105264
Violations / Measurements = 0.05567905456756346


In [4]:
# Vienna 2010-2019 simple dense
model = load_model_tflite("models/zamg_vienna_2010_2019_simple_dense")
metadata = model.metadata
data_storage = DataStorage(metadata.input_features, metadata.output_features)
data_storage.add_measurement_df(initial_df)
predictor = Predictor(model, data_storage)
predictor.update_prediction_horizon(sim_data.index.min())
model_simulator = ModelSimulator(predictor, threshold, sim_data, 600)

results_2010_2019 = model_simulator.run()
results_2010_2019

,TL,P,RF,SO,prediction,error,model,violation
2020-01-01 00:00:00,5.0,1012.4,60.0,0.0,5.1,0.1,zamg_vienna_2010_2019_simple_dense,0
2020-01-01 00:10:00,5.0,1012.5,60.0,0.0,5.032051,0.032051,zamg_vienna_2010_2019_simple_dense,0
2020-01-01 00:20:00,4.9,1012.5,61.0,0.0,4.964102,0.064102,zamg_vienna_2010_2019_simple_dense,0
2020-01-01 00:30:00,4.9,1012.5,61.0,0.0,4.896153,0.003847,zamg_vienna_2010_2019_simple_dense,0
2020-01-01 00:40:00,4.9,1012.6,60.0,0.0,4.828204,0.071796,zamg_vienna_2010_2019_simple_dense,0
...,...,...,...,...,...,...,...,...
2021-12-31 23:10:00,15.3,999.2,59.0,0.0,15.058929,0.241071,zamg_vienna_2010_2019_simple_dense,0
2021-12-31 23:20:00,15.2,999.3,59.0,0.0,14.990402,0.209598,zamg_vienna_2010_2019_simple_dense,0
2021-12-31 23:30:00,15.2,999.3,59.0,0.0,14.921875,0.278125,zamg_vienna_2010_2019_simple_dense,0
2021-12-31 23:40:00,15.0,999.7,59.0,0.0,14.853349,0.146651,zamg_vienna_2010_2019_simple_dense,0


In [5]:
violations_2010_2019 = results_2010_2019[results_2010_2019[VIOLATION_COLUMN] == True]
print(f"Simulation data: {results_2010_2019.index.min()} - {results_2010_2019.index.max()}")
print(f"Violations: {len(violations_2010_2019)}")
print(f"Measurements: {len(results_2010_2019)}")
print(f"Violations / Measurements = {len(violations_2010_2019) / len(results_2010_2019)}")

Simulation data: 2020-01-01 00:00:00 - 2021-12-31 23:50:00
Violations: 4293
Measurements: 105264
Violations / Measurements = 0.04078317373461012
